In [1]:
reload_lamb()

In [2]:
type_s = types.BasicType("s")
lang.get_system().add_basic_type(type_s)
intensional_types = meta.get_type_system()
intensional_types

# Intensional scope
### Notebook author: Kyle Rawlins / Apr 2018

This notebook goes through the basics of three approaches to scope in an intensional context. It is not intended as an introduction to the linguistic issues, but rather as a technical supplement to such an overview (which can be partially found in the von Fintel and Heim lecture notes). That is, this notebook presupposes familiarity with notions like "de re", "the third reading", etc.

The three approaches are:
* Intensionalization triggered by selection for an intensional type, binding all world variables in the scope of the selecting item (von Fintel and Heim lecture notes).
* World variables in the syntax (Percus 2000).
* Split intensionality (Keshet 2011).

I will focus on two main examples:

    (1) Joanna believes my brother is Canadian. (After one of Percus's examples)
    (2) Alfonso wants to buy an inexpensive coat. (After Keshet's version of Fodor's famous third reading example.)
    


# Scope with indices, intensionalization, and QR

We begin with the von Fintel & Heim first treatment of intensionalization. In this approach, selection of an intensional type triggers abstraction over the world of evaluation for the complement. I have implemented this here as a type shift feeding FA, rather than a separate composition operation.

In [3]:
# set up evaluation world as a parameter to the interpretation function
int_system = lang.td_system.copy()
eval_w = lang.te("w_s")
int_system.assign_controller = lang.AssignmentController(specials=[eval_w])
lang.set_system(int_system)

# code to abstract over the evaluation world.
def intension_fun(f, assignment=None):
    new_a = lang.Assignment(assignment)
    new_a.update({"w": lang.te("w_s")})
    result = meta.LFun("w_s", f.content.under_assignment(new_a))
    return lang.UnaryComposite(f, result, source="INT(%s)" % (f.name))

int_system.add_rule(lang.UnaryCompositionOp("INT", intension_fun, typeshift=True))
int_system.typeshift = True
int_system

Composition system: H&K simple (copy)

For the sake of being explicit about scope in these derivations, I will implement `my` using an existential quantifier.

In [4]:
%%lamb
||brother|| = L x_e : L y_e : Brother(x,y,w)
||canadian|| = L x_e : Canadian(x,w)
||joanna|| = Joanna_e
||my|| = L f_<e,<e,t>> : L g_<e,t> : Exists x_e : f(Speaker_e)(x) & g(x)
dox = Dox_<(e,s),{s}>
||believe|| = L p_<s,t> : L x_e : Forall w2_s : w2 << dox(x, w_s) >> p(w2)

INFO (meta): Coerced guessed type for 'Brother_t' into <(e,e,s),t>, to match argument '(x_e, y_e, w_s)'
INFO (meta): Coerced guessed type for 'Canadian_t' into <(e,s),t>, to match argument '(x_e, w_s)'


$[\![\mathbf{\text{brother}}]\!]^{{w}_{s}, g}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Brother}({x}, {y}, {w}_{s})$<br />
$[\![\mathbf{\text{canadian}}]\!]^{{w}_{s}, g}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Canadian}({x}, {w}_{s})$<br />
$[\![\mathbf{\text{joanna}}]\!]^{{w}_{s}, g}_{e} \:=\: $${Joanna}_{e}$<br />
$[\![\mathbf{\text{my}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({f}({Speaker}_{e})({x}) \wedge{} {g}({x}))$<br />
${dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}\:=\:{Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}$<br />
$[\![\mathbf{\text{believe}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}s,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} p_{\left\langle{}s,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} w2_{s} \: . \: (({w2} \in{} {Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}({x}, {w}_{s})) \rightarrow{} {p}({w2}))$

First, the basic unembedded sentence:

In [5]:
(my * brother) * canadian

CompositionResult(results=[⟦[[my brother] canadian]⟧[w_s, type s, g] = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & Canadian_<(e,s),t>(x_e, w_s)))], failures=[⟦[canadian [my brother]]⟧[w_s, type s, g] = Type mismatch: '⟦canadian⟧[w_s, type s, g] = (λ x_e: Canadian_<(e,s),t>(x_e, w_s))'/<e,t> and '⟦[my brother]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[[my brother] canadian]⟧[w_s, type s, g] = Type mismatch: '⟦[my brother]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & g_<e,t>(x_e))))'/<<e,t>,t> and '⟦canadian⟧[w_s, type s, g] = (λ x_e: Canadian_<(e,s),t>(x_e, w_s))'/<e,t> conflict (mode: Predicate Modification), ⟦[[my brother] canadian]⟧[w_s, type s, g] = Type mismatch: '⟦[my brother]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & g_<e,t>(x_e))))'/<<e,t>,t> and '⟦canadian⟧[w_s, type s, g] = (λ x_e: Canadian_<(e,s),t>(x_e, w_s))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[canadian [my brother]]⟧[w_s, type s, g] = Type mismatch: '⟦canadian⟧[w_s, type s, g] = (λ x_e: Canadian_<(e,s),t>(x_e, w_s))'/<e,t> and '⟦[my brother]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction)])

Surface scope, everything in situ:

In [6]:
(joanna * (believe * ((my * brother) * canadian))) #.tree()

CompositionResult(results=[⟦[[believe INT([[my brother] canadian])] joanna]⟧[w_s, type s, g] = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s)))))], failures=[⟦[joanna [believe INT([[my brother] canadian])]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe INT([[my brother] canadian])]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> conflict (mode: Function Application), ⟦[joanna [believe INT([[my brother] canadian])]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe INT([[my brother] canadian])]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> conflict (mode: Predicate Modification), ⟦[joanna [believe INT([[my brother] canadian])]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe INT([[my brother] canadian])]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[believe INT([[my brother] canadian])] joanna]⟧[w_s, type s, g] = Type mismatch: '⟦[believe INT([[my brother] canadian])]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> and '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e conflict (mode: Predicate Abstraction)])

In [7]:
trace = lang.Trace.index_factory(typ=types.type_e)
from lamb.lang import Binder

Wide scope (de re), DP has raised via QR (assuming this is possible across a finite clause boundary):

In [8]:
((my * brother) *
 (Binder(11) *
  (joanna * (believe *
             (trace(11) * canadian))))) #.tree()

CompositionResult(results=[⟦[[my brother] [11 [[believe INT([canadian t11])] joanna]]]⟧[w_s, type s, g] = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> Canadian_<(e,s),t>(x_e, w2_s)))))], failures=[⟦[[11 [[believe INT([canadian t11])] joanna]] [my brother]]⟧[w_s, type s, g] = Type mismatch: '⟦[11 [[believe INT([canadian t11])] joanna]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> Canadian_<(e,s),t>(x_e, w2_s))))'/<e,t> and '⟦[my brother]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[[my brother] [11 [[believe INT([canadian t11])] joanna]]]⟧[w_s, type s, g] = Type mismatch: '⟦[my brother]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & g_<e,t>(x_e))))'/<<e,t>,t> and '⟦[11 [[believe INT([canadian t11])] joanna]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> Canadian_<(e,s),t>(x_e, w2_s))))'/<e,t> conflict (mode: Predicate Modification), ⟦[[my brother] [11 [[believe INT([canadian t11])] joanna]]]⟧[w_s, type s, g] = Type mismatch: '⟦[my brother]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & g_<e,t>(x_e))))'/<<e,t>,t> and '⟦[11 [[believe INT([canadian t11])] joanna]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> Canadian_<(e,s),t>(x_e, w2_s))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[11 [[believe INT([canadian t11])] joanna]] [my brother]]⟧[w_s, type s, g] = Type mismatch: '⟦[11 [[believe INT([canadian t11])] joanna]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> Canadian_<(e,s),t>(x_e, w2_s))))'/<e,t> and '⟦[my brother]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction)])

With just these tools, there is no other way to get another reading.

# World variables

This is the barest sketch of the world variable proposal in Percus 2000, *Constraints on some other variables in syntax*, Natural Language Semantics 8.

In [9]:
w = lang.IndexedPronoun.index_factory("w", typ=type_s)

In [10]:
# back to the basic composition system, just need to generalize PA
world_binding_system = lang.td_system.copy()

from lamb.types import TypeMismatch
from lamb.lang import BinaryComposite

# Predicate abstraction for world variables. Based on lang.sbc_pa
def world_pa(binder, content, assignment=None):
    if (binder.content is not None) or not binder.name.strip().isnumeric():
        raise TypeMismatch(binder, content, "Predicate Abstraction (type s)")
    index = int(binder.name.strip())
    vname = "var%i" % index
    bound_var = meta.term(vname, type_s)
    f = meta.LFun(type_s, content.content.calculate_partiality({bound_var}), vname)
    return BinaryComposite(binder, content, f)

world_binding_system.add_rule(lang.BinaryCompositionOp("WPA", world_pa, allow_none=True))
lang.set_system(world_binding_system)
world_binding_system

Composition system: H&K simple (copy)

In [11]:
%%lamb
||brother|| = L w_s : L x_e : L y_e : Brother(x,y,w)
||canadian|| = L w_s : L x_e : Canadian(x,w)
||joanna|| = Joanna_e
||my|| = L f_<e,<e,t>> : L g_<e,t> : Exists x_e : f(Speaker_e)(x) & g(x)
dox = Dox_<(e,s),{s}>
||believe|| = L w_s : L p_<s,t> : L x_e : Forall w2_s : w2 << dox(x, w_s) >> p(w2)

INFO (meta): Coerced guessed type for 'Brother_t' into <(e,e,s),t>, to match argument '(x_e, y_e, w_s)'
INFO (meta): Coerced guessed type for 'Canadian_t' into <(e,s),t>, to match argument '(x_e, w_s)'


$[\![\mathbf{\text{brother}}]\!]^{}_{\left\langle{}s,\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} w_{s} \: . \: \lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Brother}({x}, {y}, {w})$<br />
$[\![\mathbf{\text{canadian}}]\!]^{}_{\left\langle{}s,\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} w_{s} \: . \: \lambda{} x_{e} \: . \: {Canadian}({x}, {w})$<br />
$[\![\mathbf{\text{joanna}}]\!]^{}_{e} \:=\: $${Joanna}_{e}$<br />
$[\![\mathbf{\text{my}}]\!]^{}_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({f}({Speaker}_{e})({x}) \wedge{} {g}({x}))$<br />
${dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}\:=\:{Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}$<br />
$[\![\mathbf{\text{believe}}]\!]^{}_{\left\langle{}s,\left\langle{}\left\langle{}s,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} w_{s} \: . \: \lambda{} p_{\left\langle{}s,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} w2_{s} \: . \: (({w2} \in{} {Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}({x}, {w})) \rightarrow{} {p}({w2}))$

Basic unembedded sentence. As long as all world variables must be bound, and binders are only inserted up to interpretability to achieve a type $\langle s,t \rangle$ at the root, this is the only possible binding configuration for this sentence.

In [12]:
Binder(1) * ((my * (brother * w(1))) * (canadian * w(1)))

CompositionResult(results=[⟦[1 [[my [brother w1]] [canadian w1]]]⟧ = (λ var1_s: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var1_s) & Canadian_<(e,s),t>(x_e, var1_s))))], failures=[⟦[1 [[my [brother w1]] [canadian w1]]]⟧ = Type mismatch: '⟦1⟧ = None' and '⟦[[my [brother w1]] [canadian w1]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var1_s) & Canadian_<(e,s),t>(x_e, var1_s)))'/t conflict (mode: FA), ⟦[[[my [brother w1]] [canadian w1]] 1]⟧ = Type mismatch: '⟦[[my [brother w1]] [canadian w1]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var1_s) & Canadian_<(e,s),t>(x_e, var1_s)))'/t and '⟦1⟧ = None' conflict (mode: FA), ⟦[1 [[my [brother w1]] [canadian w1]]]⟧ = Type mismatch: '⟦1⟧ = None' and '⟦[[my [brother w1]] [canadian w1]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var1_s) & Canadian_<(e,s),t>(x_e, var1_s)))'/t conflict (mode: PM), ⟦[1 [[my [brother w1]] [canadian w1]]]⟧ = Type mismatch: 'var1_s'/s and 'var1_e'/e conflict (mode: Variable replace (unify failed)), ⟦[[[my [brother w1]] [canadian w1]] 1]⟧ = Type mismatch: '⟦[[my [brother w1]] [canadian w1]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var1_s) & Canadian_<(e,s),t>(x_e, var1_s)))'/t and '⟦1⟧ = None' conflict (mode: Predicate Abstraction), ⟦[[[my [brother w1]] [canadian w1]] 1]⟧ = Type mismatch: '⟦[[my [brother w1]] [canadian w1]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var1_s) & Canadian_<(e,s),t>(x_e, var1_s)))'/t and '⟦1⟧ = None' conflict (mode: Predicate Abstraction (type s))])

Pure de dicto sentence:

In [13]:
r = Binder(5) * (joanna *
                 ((believe * w(5)) *
                  (Binder(1) * ((my * (brother * w(1))) * (canadian * w(1))))))
r

CompositionResult(results=[⟦[5 [[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]]⟧ = (λ var5_s: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s))))))], failures=[⟦[5 [[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]]⟧ = Type mismatch: '⟦5⟧ = None' and '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t conflict (mode: FA), ⟦[[[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: FA), ⟦[5 [[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]]⟧ = Type mismatch: '⟦5⟧ = None' and '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t conflict (mode: PM), ⟦[5 [[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]]⟧ = Type mismatch: 'var5_s'/s and 'var5_e'/e conflict (mode: Variable replace (unify failed)), ⟦[[[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: Predicate Abstraction), ⟦[[[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: Predicate Abstraction (type s))])

In [14]:
r.tree()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{5}}]\!]^{}$</div><div style="vertical-align:bottom;text-align:center">N/A</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{believe}}]\!]^{}_{\left\langle{}s,\left\langle{}\left\langle{}s,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} w_{s} \: . \: \lambda{} p_{\left\langle{}s,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} w2_{s} \: . \: (({w2} \in{} {Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}({x}, {w})) \rightarrow{} {p}({w2}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{w}}_{5}]\!]^{}_{s}$</div><div style="vertical-align:bottom;text-align:center">${var5}_{s}$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[believe w5]}}]\!]^{}_{\left\langle{}\left\langle{}s,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} p_{\left\langle{}s,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} w2_{s} \: . \: (({w2} \in{} {Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}({x}, {var5}_{s})) \rightarrow{} {p}({w2}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{1}}]\!]^{}$</div><div style="vertical-align:bottom;text-align:center">N/A</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;

In principle there are several other binding configurations that are possible even without QR. Let's also assume a constraint against vacuous binding, with the effect that `Binder(1)` being inserted (required for semantic composition) forces at least one bound variable downstairs.

In [15]:
r = Binder(5) * (joanna * ((believe * w(5)) *
                           (Binder(1) *
                            ((my * (brother * w(5))) * (canadian * w(1))))))
r

CompositionResult(results=[⟦[5 [[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]]⟧ = (λ var5_s: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & Canadian_<(e,s),t>(x_e, w2_s))))))], failures=[⟦[5 [[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]]⟧ = Type mismatch: '⟦5⟧ = None' and '⟦[[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t conflict (mode: FA), ⟦[[[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: FA), ⟦[5 [[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]]⟧ = Type mismatch: '⟦5⟧ = None' and '⟦[[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t conflict (mode: PM), ⟦[5 [[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]]⟧ = Type mismatch: 'var5_s'/s and 'var5_e'/e conflict (mode: Variable replace (unify failed)), ⟦[[[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: Predicate Abstraction), ⟦[[[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w5]] [canadian w1]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: Predicate Abstraction (type s))])

In [16]:
r = Binder(5) * (joanna * ((believe * w(5)) * (Binder(1) * ((my * (brother * w(1))) * (canadian * w(5))))))
r

CompositionResult(results=[⟦[5 [[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]]⟧ = (λ var5_s: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, var5_s))))))], failures=[⟦[5 [[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]]⟧ = Type mismatch: '⟦5⟧ = None' and '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, var5_s)))))'/t conflict (mode: FA), ⟦[[[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, var5_s)))))'/t and '⟦5⟧ = None' conflict (mode: FA), ⟦[5 [[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]]⟧ = Type mismatch: '⟦5⟧ = None' and '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, var5_s)))))'/t conflict (mode: PM), ⟦[5 [[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]]⟧ = Type mismatch: 'var5_s'/s and 'var5_e'/e conflict (mode: Variable replace (unify failed)), ⟦[[[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, var5_s)))))'/t and '⟦5⟧ = None' conflict (mode: Predicate Abstraction), ⟦[[[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna] 5]⟧ = Type mismatch: '⟦[[[believe w5] [1 [[my [brother w1]] [canadian w5]]]] joanna]⟧ = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s) & Canadian_<(e,s),t>(x_e, var5_s)))))'/t and '⟦5⟧ = None' conflict (mode: Predicate Abstraction (type s))])

The second of these binding configurations would be banned by Percus's *Generalization X*. A slightly reformulated version of this condition: the world variable that a VP predicate selects most be coindexed with the nearest c-commanding binder.

With QR we would get another possibility. This binding configuration is really the only possibility for this scope configuration.

In [17]:
r = Binder(5) * ((my * (brother * w(5))) *
                 (Binder(3) * (joanna * ((believe * w(5)) *
                                         (Binder(1) *
                                          (trace(3) * (canadian * w(1))))))))
r

CompositionResult(results=[⟦[5 [[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]]⟧ = (λ var5_s: (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> Canadian_<(e,s),t>(x_e, w2_s))))))], failures=[⟦[5 [[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]]⟧ = Type mismatch: '⟦5⟧ = None' and '⟦[[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> Canadian_<(e,s),t>(x_e, w2_s)))))'/t conflict (mode: FA), ⟦[[[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]] 5]⟧ = Type mismatch: '⟦[[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: FA), ⟦[5 [[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]]⟧ = Type mismatch: '⟦5⟧ = None' and '⟦[[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> Canadian_<(e,s),t>(x_e, w2_s)))))'/t conflict (mode: PM), ⟦[5 [[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]]⟧ = Type mismatch: 'var5_s'/s and 'var5_e'/e conflict (mode: Variable replace (unify failed)), ⟦[[[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]] 5]⟧ = Type mismatch: '⟦[[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: Predicate Abstraction), ⟦[[[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]] 5]⟧ = Type mismatch: '⟦[[my [brother w5]] [3 [[[believe w5] [1 [[canadian w1] t3]]] joanna]]]⟧ = (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, var5_s) & (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, var5_s)) >> Canadian_<(e,s),t>(x_e, w2_s)))))'/t and '⟦5⟧ = None' conflict (mode: Predicate Abstraction (type s))])

# Split intensionality

This approach illustrates the core of the analysis in Keshet 2011, *Split intensionality: a new scope theory of de re and de dicto*, Linguistics and Philosophy 33. I will set things up slightly differently, implementing his intensionalization operation as a unary composition operation, rather than an operator. (This is mainly because it is a bit more straightforward in the lambda notebook, rather than for any deep reasons.)

In [18]:
# set up evaluation world as a parameter to the interpretation function
int_system = lang.td_system.copy()
eval_w = lang.te("w_s")
int_system.assign_controller = lang.AssignmentController(specials=[eval_w])
lang.set_system(int_system)

# same intensionalization code as in the first version
def intension_fun(f, assignment=None):
    new_a = lang.Assignment(assignment)
    new_a.update({"w": lang.te("w_s")})
    result = meta.LFun("w_s", f.content.under_assignment(new_a))
    return lang.UnaryComposite(f, result, source="INT(%s)" % (f.name))

# add INT as a unary operation, *not* a type shift.
int_system.add_rule(lang.UnaryCompositionOp("INT", intension_fun))
int_system

Composition system: H&K simple (copy)

There are two key ideas: (i) instead of generating propositions via a world binder inserted up to interpretability, we generate propositions with a shifting intensionalization operator (I will call this operator `INT`, Keshet notates it as ${}^{\wedge}$ after Montague). (ii) this intensionalization operator has syntactic presence, and QR can target a position above the operator or below it. The position above it generates transparent non-specific readings, but doesn't involve movement across a clause boundary.

In [19]:
%%lamb
||brother|| = L x_e : L y_e : Brother(x,y,w)
||canadian|| = L x_e : Canadian(x,w)
||joanna|| = Joanna_e
||my|| = L f_<e,<e,t>> : Iota x_e : f(Speaker_e)(x) # note change from above -- discussed below
dox = Dox_<(e,s),{s}>
||believe|| = L p_<s,t> : L x_e : Forall w2_s : w2 << dox(x, w_s) >> p(w2)

INFO (meta): Coerced guessed type for 'Brother_t' into <(e,e,s),t>, to match argument '(x_e, y_e, w_s)'
INFO (meta): Coerced guessed type for 'Canadian_t' into <(e,s),t>, to match argument '(x_e, w_s)'


$[\![\mathbf{\text{brother}}]\!]^{{w}_{s}, g}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Brother}({x}, {y}, {w}_{s})$<br />
$[\![\mathbf{\text{canadian}}]\!]^{{w}_{s}, g}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Canadian}({x}, {w}_{s})$<br />
$[\![\mathbf{\text{joanna}}]\!]^{{w}_{s}, g}_{e} \:=\: $${Joanna}_{e}$<br />
$[\![\mathbf{\text{my}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},e\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \: . \: \iota{} x_{e} \: . \: {f}({Speaker}_{e})({x})$<br />
${dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}\:=\:{Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}$<br />
$[\![\mathbf{\text{believe}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}s,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} p_{\left\langle{}s,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} w2_{s} \: . \: (({w2} \in{} {Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}({x}, {w}_{s})) \rightarrow{} {p}({w2}))$

The basic case is much the same as on the starting approach. To trigger unary composition, compose with `None`:

In [20]:
((my * brother) * canadian) * None

CompositionResult(results=[⟦INT([canadian [my brother]])⟧[w_s, type s, g] = (λ w_s: Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s)), w_s))], failures=[])

The pure de dicto reading is not especially new or exciting; where before there was a type-shift, now we have an intensionalization operator:

In [21]:
r = (joanna * (believe *
               (((my * brother) * canadian) * None)))
r

CompositionResult(results=[⟦[[believe INT([canadian [my brother]])] joanna]⟧[w_s, type s, g] = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s)), w2_s)))], failures=[⟦[joanna [believe INT([canadian [my brother]])]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe INT([canadian [my brother]])]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s)), w2_s))))'/<e,t> conflict (mode: Function Application), ⟦[joanna [believe INT([canadian [my brother]])]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe INT([canadian [my brother]])]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s)), w2_s))))'/<e,t> conflict (mode: Predicate Modification), ⟦[joanna [believe INT([canadian [my brother]])]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe INT([canadian [my brother]])]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s)), w2_s))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[believe INT([canadian [my brother]])] joanna]⟧[w_s, type s, g] = Type mismatch: '⟦[believe INT([canadian [my brother]])]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w2_s)), w2_s))))'/<e,t> and '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e conflict (mode: Predicate Abstraction)])

In [22]:
r.tree()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{believe}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}s,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} p_{\left\langle{}s,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} w2_{s} \: . \: (({w2} \in{} {Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}({x}, {w}_{s})) \rightarrow{} {p}({w2}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{canadian}}]\!]^{{w}_{s}, g}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Canadian}({x}, {w}_{s})$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{my}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},e\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \: . \: \iota{} x_{e} \: . \: {f}({Speaker}_{e})({x})$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{brother}}]\!]^{{w}_{s}, g}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Brother}({x}, {y}, {w}_{s})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[my brother]}}]\!]^{{w}_{s}, g}_{e}$</div><div style="vertical-align:bottom;text-align:center">$\iota{} x_{e} \: . \: {Brother}({Speaker}_{e}, {x}, {w}_{s})$</div></div></td><td></td></tr></table></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padd

What is new is the intermediate QR derivation:

In [23]:
r = (joanna * (believe *
               ((my * brother) * (Binder(3) *
                                  ((trace(3) * canadian) * None)))))
r

CompositionResult(results=[⟦[[believe [[3 INT([canadian t3])] [my brother]]] joanna]⟧[w_s, type s, g] = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s)), w2_s)))], failures=[⟦[joanna [believe [[3 INT([canadian t3])] [my brother]]]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe [[3 INT([canadian t3])] [my brother]]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s)), w2_s))))'/<e,t> conflict (mode: Function Application), ⟦[joanna [believe [[3 INT([canadian t3])] [my brother]]]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe [[3 INT([canadian t3])] [my brother]]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s)), w2_s))))'/<e,t> conflict (mode: Predicate Modification), ⟦[joanna [believe [[3 INT([canadian t3])] [my brother]]]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe [[3 INT([canadian t3])] [my brother]]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s)), w2_s))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[believe [[3 INT([canadian t3])] [my brother]]] joanna]⟧[w_s, type s, g] = Type mismatch: '⟦[believe [[3 INT([canadian t3])] [my brother]]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s)), w2_s))))'/<e,t> and '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e conflict (mode: Predicate Abstraction)])

And finally, the pure de re reading. There is no way to generate the reading that Percus needed to ban with generalization X, because intensional scope is still parasitic on DP scope.

In [24]:
((my * brother) * (Binder(3) *
                   ((trace(3) * canadian) * None)))

CompositionResult(results=[⟦[[3 INT([canadian t3])] [my brother]]⟧[w_s, type s, g] = (λ w1_s: Canadian_<(e,s),t>((ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s)), w1_s))], failures=[⟦[[my brother] [3 INT([canadian t3])]]⟧[w_s, type s, g] = Type mismatch: '⟦[my brother]⟧[w_s, type s, g] = (ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s))'/e and '⟦[3 INT([canadian t3])]⟧[w_s, type s, g] = (λ x_e: (λ w_s: Canadian_<(e,s),t>(x_e, w_s)))'/<e,<s,t>> conflict (mode: Function Application), ⟦[[my brother] [3 INT([canadian t3])]]⟧[w_s, type s, g] = Type mismatch: '⟦[my brother]⟧[w_s, type s, g] = (ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s))'/e and '⟦[3 INT([canadian t3])]⟧[w_s, type s, g] = (λ x_e: (λ w_s: Canadian_<(e,s),t>(x_e, w_s)))'/<e,<s,t>> conflict (mode: Predicate Modification), ⟦[[my brother] [3 INT([canadian t3])]]⟧[w_s, type s, g] = Type mismatch: '⟦[my brother]⟧[w_s, type s, g] = (ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s))'/e and '⟦[3 INT([canadian t3])]⟧[w_s, type s, g] = (λ x_e: (λ w_s: Canadian_<(e,s),t>(x_e, w_s)))'/<e,<s,t>> conflict (mode: Predicate Abstraction), ⟦[[3 INT([canadian t3])] [my brother]]⟧[w_s, type s, g] = Type mismatch: '⟦[3 INT([canadian t3])]⟧[w_s, type s, g] = (λ x_e: (λ w_s: Canadian_<(e,s),t>(x_e, w_s)))'/<e,<s,t>> and '⟦[my brother]⟧[w_s, type s, g] = (ι x_e: Brother_<(e,e,s),t>(Speaker_e, x_e, w_s))'/e conflict (mode: Predicate Abstraction)])

One complication is that a regular generalized quantifier type won't work with regular FA on this approach, if it scopes above `INT`. You can see this by examining the type generated for a binder scoping over `INT`:

In [25]:
Binder(3) * ((trace(3) * canadian) * None)

CompositionResult(results=[⟦[3 INT([canadian t3])]⟧[w_s, type s, g] = (λ x_e: (λ w_s: Canadian_<(e,s),t>(x_e, w_s)))], failures=[⟦[3 INT([canadian t3])]⟧[w_s, type s, g] = Type mismatch: '⟦3⟧[w_s, type s, g] = None' and '⟦INT([canadian t3])⟧[w_s, type s, g] = (λ w_s: Canadian_<(e,s),t>(var3_e, w_s))'/<s,t> conflict (mode: FA), ⟦[INT([canadian t3]) 3]⟧[w_s, type s, g] = Type mismatch: '⟦INT([canadian t3])⟧[w_s, type s, g] = (λ w_s: Canadian_<(e,s),t>(var3_e, w_s))'/<s,t> and '⟦3⟧[w_s, type s, g] = None' conflict (mode: FA), ⟦[3 INT([canadian t3])]⟧[w_s, type s, g] = Type mismatch: '⟦3⟧[w_s, type s, g] = None' and '⟦INT([canadian t3])⟧[w_s, type s, g] = (λ w_s: Canadian_<(e,s),t>(var3_e, w_s))'/<s,t> conflict (mode: PM), ⟦[INT([canadian t3]) 3]⟧[w_s, type s, g] = Type mismatch: '⟦INT([canadian t3])⟧[w_s, type s, g] = (λ w_s: Canadian_<(e,s),t>(var3_e, w_s))'/<s,t> and '⟦3⟧[w_s, type s, g] = None' conflict (mode: Predicate Abstraction)])

That is, an (extensional) generalized quantifier DP will be type $\langle \langle e,t \rangle , t \rangle$, but we really need something of type $\langle \langle e,\langle s,t\rangle \rangle , \langle s,t\rangle \rangle$. Keshet proposes a solution to this using a different composition operation for generalized quantifiers due to Daniel Buring; this operation also allows composition in object position without QR. This operation is rather complex and general, but for dealing with `INT` we really only need one special case, which I implement here as a binary composition via the following combinator. 'AS' stands for Argument Saturation.

In [26]:
as_combinator = %te L quant_<<e,t>,t> : L f_<e,<X,t>> : L y_X : quant(L x_e : f(x)(y))
as_combinator

(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))

In [27]:
#int_system.add_typeshift(as_combinator, "AS")
#int_system.typeshift = True
int_system.add_binary_rule(as_combinator, "AS")
int_system

Composition system: H&K simple (copy)

In [28]:
%%lamb
||my|| = L f_<e,<e,t>> : L g_<e,t> : Exists x_e : f(Speaker_e)(x) & g(x)

$[\![\mathbf{\text{my}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({f}({Speaker}_{e})({x}) \wedge{} {g}({x}))$

In [29]:
r = (joanna * (believe *
               ((my * brother) * (Binder(3) *
                                  ((trace(3) * canadian) * None)))))
r

CompositionResult(results=[⟦[[believe [[my brother] [3 INT([canadian t3])]]] joanna]⟧[w_s, type s, g] = (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(Joanna_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & Canadian_<(e,s),t>(x_e, w2_s)))))], failures=[⟦[joanna [believe [[my brother] [3 INT([canadian t3])]]]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe [[my brother] [3 INT([canadian t3])]]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> conflict (mode: Function Application), ⟦[joanna [believe [[my brother] [3 INT([canadian t3])]]]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe [[my brother] [3 INT([canadian t3])]]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> conflict (mode: Predicate Modification), ⟦[joanna [believe [[my brother] [3 INT([canadian t3])]]]]⟧[w_s, type s, g] = Type mismatch: '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e and '⟦[believe [[my brother] [3 INT([canadian t3])]]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[believe [[my brother] [3 INT([canadian t3])]]] joanna]⟧[w_s, type s, g] = Type mismatch: '⟦[believe [[my brother] [3 INT([canadian t3])]]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> and '⟦joanna⟧[w_s, type s, g] = Joanna_e'/e conflict (mode: Predicate Abstraction), ⟦[joanna [believe [[my brother] [3 INT([canadian t3])]]]]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and 'Joanna_e'/e conflict (mode: Function argument combination (unification failed)), ⟦[[believe [[my brother] [3 INT([canadian t3])]]] joanna]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and '(λ x_e: (Forall w2_s: ((w2_s << Dox_<(e,s),{s}>(x_e, w_s)) >> (Exists x_e: (Brother_<(e,e,s),t>(Speaker_e, x_e, w_s) & Canadian_<(e,s),t>(x_e, w2_s))))))'/<e,t> conflict (mode: Function argument combination (unification failed))])

In [30]:
r.tree()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{believe}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}s,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} p_{\left\langle{}s,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} w2_{s} \: . \: (({w2} \in{} {Dox}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}({x}, {w}_{s})) \rightarrow{} {p}({w2}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{my}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({f}({Speaker}_{e})({x}) \wedge{} {g}({x}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{brother}}]\!]^{{w}_{s}, g}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Brother}({x}, {y}, {w}_{s})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[my brother]}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({Brother}({Speaker}_{e}, {x}, {w}_{s}) \wedge{} {g}({x}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{3}}]\!]^{{w}_{s}, g}$</div><div style="vertical-align:bottom;text-align:center">N/A</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$<

In any case where QR above the intensional lexical item is possible, Keshet's system generates Fodor's three readings: a pure de dicto reading (non-specific/opaque), a pure de re reading (specific/transparent), and the third reading (non-specific/transparent). Here are the readings in an example more tailored to have them:

In [31]:
%%lamb
||buy|| = L x_e : L y_e : Buy(y,x,w)
||coat|| = L x_e : Coat(x,w)
||inexpensive|| = L x_e : Inexpensive(x,w)
||want|| = L p_<s,t> : L x_e  : Forall w2_s : w2 << Des_<(e,s),{s}>(x,w) >> p(w2)
||alfonso|| = Alfonso_e
||an|| = L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)

INFO (meta): Coerced guessed type for 'Buy_t' into <(e,e,s),t>, to match argument '(y_e, x_e, w_s)'
INFO (meta): Coerced guessed type for 'Coat_t' into <(e,s),t>, to match argument '(x_e, w_s)'
INFO (meta): Coerced guessed type for 'Inexpensive_t' into <(e,s),t>, to match argument '(x_e, w_s)'


$[\![\mathbf{\text{buy}}]\!]^{{w}_{s}, g}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Buy}({y}, {x}, {w}_{s})$<br />
$[\![\mathbf{\text{coat}}]\!]^{{w}_{s}, g}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Coat}({x}, {w}_{s})$<br />
$[\![\mathbf{\text{inexpensive}}]\!]^{{w}_{s}, g}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Inexpensive}({x}, {w}_{s})$<br />
$[\![\mathbf{\text{want}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}s,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} p_{\left\langle{}s,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} w2_{s} \: . \: (({w2} \in{} {Des}_{\left\langle{}\left(e, s\right),\left\{s\right\}\right\rangle{}}({x}, {w}_{s})) \rightarrow{} {p}({w2}))$<br />
$[\![\mathbf{\text{alfonso}}]\!]^{{w}_{s}, g}_{e} \:=\: $${Alfonso}_{e}$<br />
$[\![\mathbf{\text{an}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({f}({x}) \wedge{} {g}({x}))$

In [32]:
PRO = lang.IndexedPronoun.index_factory("PRO")

Pure de dicto (non-specific / opaque), quantifier scopes under `INT`:

In [33]:
r = alfonso * (Binder(10) *
               (trace(10) * (want *
                             (((an * (inexpensive * coat)) * (Binder(3) *
                                                              (PRO(10) * (buy * trace(3)))))
                              * None))))
r

CompositionResult(results=[⟦[[10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]] alfonso]⟧[w_s, type s, g] = (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s)))))], failures=[⟦[alfonso [10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]]]⟧[w_s, type s, g] = Type mismatch: '⟦alfonso⟧[w_s, type s, g] = Alfonso_e'/e and '⟦[10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> conflict (mode: Function Application), ⟦[alfonso [10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]]]⟧[w_s, type s, g] = Type mismatch: '⟦alfonso⟧[w_s, type s, g] = Alfonso_e'/e and '⟦[10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> conflict (mode: Predicate Modification), ⟦[alfonso [10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]]]⟧[w_s, type s, g] = Type mismatch: '⟦alfonso⟧[w_s, type s, g] = Alfonso_e'/e and '⟦[10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]] alfonso]⟧[w_s, type s, g] = Type mismatch: '⟦[10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> and '⟦alfonso⟧[w_s, type s, g] = Alfonso_e'/e conflict (mode: Predicate Abstraction), ⟦[alfonso [10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]]]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and 'Alfonso_e'/e conflict (mode: Function argument combination (unification failed)), ⟦[[10 [[want INT([[an [inexpensive coat]] [3 [[buy t3] PRO10]]])] t10]] alfonso]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and '(λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> conflict (mode: Function argument combination (unification failed))])

Mixed (non-specific / transparent), quantifier scopes over `INT` but within clause:

In [34]:
r = alfonso * (Binder(10) *
               (trace(10) * (want *
                             ((an * (inexpensive * coat)) * (Binder(3) *
                                                             ((PRO(10) * (buy * trace(3)))
                                                              * None))))))
r

CompositionResult(results=[⟦[[10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]] alfonso]⟧[w_s, type s, g] = (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s)))))], failures=[⟦[alfonso [10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]]]⟧[w_s, type s, g] = Type mismatch: '⟦alfonso⟧[w_s, type s, g] = Alfonso_e'/e and '⟦[10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> conflict (mode: Function Application), ⟦[alfonso [10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]]]⟧[w_s, type s, g] = Type mismatch: '⟦alfonso⟧[w_s, type s, g] = Alfonso_e'/e and '⟦[10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> conflict (mode: Predicate Modification), ⟦[alfonso [10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]]]⟧[w_s, type s, g] = Type mismatch: '⟦alfonso⟧[w_s, type s, g] = Alfonso_e'/e and '⟦[10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]] alfonso]⟧[w_s, type s, g] = Type mismatch: '⟦[10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> and '⟦alfonso⟧[w_s, type s, g] = Alfonso_e'/e conflict (mode: Predicate Abstraction), ⟦[alfonso [10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]]]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and 'Alfonso_e'/e conflict (mode: Function argument combination (unification failed)), ⟦[[10 [[want [[an [inexpensive coat]] [3 INT([[buy t3] PRO10])]]] t10]] alfonso]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and '(λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(x1_e, w_s)) >> (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & Buy_<(e,e,s),t>(x1_e, x_e, w2_s))))))'/<e,t> conflict (mode: Function argument combination (unification failed))])

Pure de re (specific/transparent), quantifier QRs all the way out across the non-finite clause boundary:

In [35]:
r = ((an * (inexpensive * coat)) * (Binder(3) *
                                    (alfonso * (Binder(10) * (trace(10) *
                                                              (want *
                                                               ((PRO(10) * (buy * trace(3)))
                                                                * None)))))))
r

CompositionResult(results=[⟦[[an [inexpensive coat]] [3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]]]⟧[w_s, type s, g] = (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s)))))], failures=[⟦[[3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]] [an [inexpensive coat]]]⟧[w_s, type s, g] = Type mismatch: '⟦[3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s))))'/<e,t> and '⟦[an [inexpensive coat]]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[[an [inexpensive coat]] [3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '⟦[an [inexpensive coat]]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & g_<e,t>(x_e))))'/<<e,t>,t> and '⟦[3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s))))'/<e,t> conflict (mode: Predicate Modification), ⟦[[an [inexpensive coat]] [3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '⟦[an [inexpensive coat]]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & g_<e,t>(x_e))))'/<<e,t>,t> and '⟦[3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]] [an [inexpensive coat]]]⟧[w_s, type s, g] = Type mismatch: '⟦[3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]]⟧[w_s, type s, g] = (λ x_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s))))'/<e,t> and '⟦[an [inexpensive coat]]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction), ⟦[[an [inexpensive coat]] [3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '((λ quant_<<e,t>,t>: (λ f_<e,<I799,t>>: (λ y_I799: quant_<<e,t>,t>((λ x_e: f_<e,<I799,t>>(x_e)(y_I799)))))))((λ g_<e,t>: (Exists x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & g_<e,t>(x_e)))))'/<<e,<I799,t>>,<I799,t>> and '(λ x_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s))))'/<e,t> conflict (mode: Function argument combination (unification failed)), ⟦[[3 [[10 [[want INT([[buy t3] PRO10])] t10]] alfonso]] [an [inexpensive coat]]]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and '(λ x_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, x_e, w2_s))))'/<e,t> conflict (mode: Function argument combination (unification failed))])

# Fourth readings

None of the systems above generate the so-called fourth reading: specific/opaque. This is because with QR as the scope mechanism, the QR target position provides a lower bound on the world binding, and this kind of reading would require a world binding that is local, but a quantifier position that is not. There has been some debate about whether the fourth reading exists, so perhaps this is a virtue: see Keshet 2011 footnotes 4,15 for an argument along these lines. However, Szabo in a 2010 Amsterdam Colloquium paper (*Specific, yet opaque*) provides fairly convincing cases of the fourth reading.

If you do want to derive this reading, there are various options.
* Szabo: add a new QR operation that moves only the determiner.
* Do partial reconstruction of the NP from a pure de re reading into a position where the world variable for the NP gets locally bound. (For Percus, under the world binder, for Keshet or a type-shifting approach, under the `INT` operator.)

Both of these are a bit convoluted. Here's a sketch of what Szabo's new QR operation would do. This operation moves a quantifier with a silent `thing` restrictor, and leaves a trace that gets interpreted via a variant (of sorts) of Fox's Trace Conversion rule. In Szabo's system, there is a specialized composition rule for this, but for the sake of simplicity, I just hard-code this in a specialized pre-indexed trace entry `rt3`.

Unfortunately some of the simplification involved in this use of Iota is not currently implemented in the lambda notebook.

In [36]:
%%lamb
||thing|| = L x_e : True
||rt3|| = L f_<e,t> : Iota x_e: f(x) & x==var3

$[\![\mathbf{\text{thing}}]\!]^{{w}_{s}, g}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {True}_{t}$<br />
$[\![\mathbf{\text{rt3}}]\!]^{{w}_{s}, g}_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \iota{} x_{e} \: . \: ({f}({x}) \wedge{} ({x} = {var3}_{e}))$

First, it may be helpful to see a derivation without an intensional lexical item where this rule has applied. In the result, the formula will be false for any potential witnesses for the existential that don't verify the properties under the $\iota$ operator, leading to equivalence with the standard truth-conditions. (This is not something that the lambda notebook knows how to simplify for you.)

In [37]:
((an * thing) * (Binder(3) * (alfonso * (buy * (rt3 * (inexpensive * coat))))))

CompositionResult(results=[⟦[[an thing] [3 [[buy [rt3 [inexpensive coat]]] alfonso]]]⟧[w_s, type s, g] = (Exists x_e: Buy_<(e,e,s),t>(Alfonso_e, (ι x2_e: ((Inexpensive_<(e,s),t>(x2_e, w_s) & Coat_<(e,s),t>(x2_e, w_s)) & (x2_e <=> x_e))), w_s))], failures=[⟦[[3 [[buy [rt3 [inexpensive coat]]] alfonso]] [an thing]]⟧[w_s, type s, g] = Type mismatch: '⟦[3 [[buy [rt3 [inexpensive coat]]] alfonso]]⟧[w_s, type s, g] = (λ x1_e: Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & (x_e <=> x1_e))), w_s))'/<e,t> and '⟦[an thing]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: g_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function Application), ⟦[[an thing] [3 [[buy [rt3 [inexpensive coat]]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '⟦[an thing]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: g_<e,t>(x_e)))'/<<e,t>,t> and '⟦[3 [[buy [rt3 [inexpensive coat]]] alfonso]]⟧[w_s, type s, g] = (λ x1_e: Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & (x_e <=> x1_e))), w_s))'/<e,t> conflict (mode: Predicate Modification), ⟦[[an thing] [3 [[buy [rt3 [inexpensive coat]]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '⟦[an thing]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: g_<e,t>(x_e)))'/<<e,t>,t> and '⟦[3 [[buy [rt3 [inexpensive coat]]] alfonso]]⟧[w_s, type s, g] = (λ x1_e: Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & (x_e <=> x1_e))), w_s))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[3 [[buy [rt3 [inexpensive coat]]] alfonso]] [an thing]]⟧[w_s, type s, g] = Type mismatch: '⟦[3 [[buy [rt3 [inexpensive coat]]] alfonso]]⟧[w_s, type s, g] = (λ x1_e: Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & (x_e <=> x1_e))), w_s))'/<e,t> and '⟦[an thing]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: g_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Predicate Abstraction), ⟦[[an thing] [3 [[buy [rt3 [inexpensive coat]]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '((λ quant_<<e,t>,t>: (λ f_<e,<I872,t>>: (λ y_I872: quant_<<e,t>,t>((λ x_e: f_<e,<I872,t>>(x_e)(y_I872)))))))((λ g_<e,t>: (Exists x_e: g_<e,t>(x_e))))'/<<e,<I872,t>>,<I872,t>> and '(λ x1_e: Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & (x_e <=> x1_e))), w_s))'/<e,t> conflict (mode: Function argument combination (unification failed)), ⟦[[3 [[buy [rt3 [inexpensive coat]]] alfonso]] [an thing]]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and '(λ x1_e: Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w_s) & Coat_<(e,s),t>(x_e, w_s)) & (x_e <=> x1_e))), w_s))'/<e,t> conflict (mode: Function argument combination (unification failed))])

Here is a fourth reading derivation, using Keshet's intensionalization operator from above.

In [38]:
((an * thing) * (Binder(3) * (alfonso * (Binder(10) *
                                         (trace(10) * (want * ((PRO(10) *
                                                               (buy * (rt3 * (inexpensive * coat))))
                                                               * None)))))))

CompositionResult(results=[⟦[[an thing] [3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]]]⟧[w_s, type s, g] = (Exists x_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, (ι x2_e: ((Inexpensive_<(e,s),t>(x2_e, w2_s) & Coat_<(e,s),t>(x2_e, w2_s)) & (x2_e <=> x_e))), w2_s))))], failures=[⟦[[3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]] [an thing]]⟧[w_s, type s, g] = Type mismatch: '⟦[3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & (x_e <=> x1_e))), w2_s))))'/<e,t> and '⟦[an thing]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: g_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function Application), ⟦[[an thing] [3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '⟦[an thing]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: g_<e,t>(x_e)))'/<<e,t>,t> and '⟦[3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & (x_e <=> x1_e))), w2_s))))'/<e,t> conflict (mode: Predicate Modification), ⟦[[an thing] [3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '⟦[an thing]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: g_<e,t>(x_e)))'/<<e,t>,t> and '⟦[3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & (x_e <=> x1_e))), w2_s))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]] [an thing]]⟧[w_s, type s, g] = Type mismatch: '⟦[3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]]⟧[w_s, type s, g] = (λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & (x_e <=> x1_e))), w2_s))))'/<e,t> and '⟦[an thing]⟧[w_s, type s, g] = (λ g_<e,t>: (Exists x_e: g_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Predicate Abstraction), ⟦[[an thing] [3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]]]⟧[w_s, type s, g] = Type mismatch: '((λ quant_<<e,t>,t>: (λ f_<e,<I974,t>>: (λ y_I974: quant_<<e,t>,t>((λ x_e: f_<e,<I974,t>>(x_e)(y_I974)))))))((λ g_<e,t>: (Exists x_e: g_<e,t>(x_e))))'/<<e,<I974,t>>,<I974,t>> and '(λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & (x_e <=> x1_e))), w2_s))))'/<e,t> conflict (mode: Function argument combination (unification failed)), ⟦[[3 [[10 [[want INT([[buy [rt3 [inexpensive coat]]] PRO10])] t10]] alfonso]] [an thing]]⟧[w_s, type s, g] = Type mismatch: '(λ quant_<<e,t>,t>: (λ f_<e,<X,t>>: (λ y_X: quant_<<e,t>,t>((λ x_e: f_<e,<X,t>>(x_e)(y_X))))))'/<<<e,t>,t>,<<e,<X,t>>,<X,t>>> and '(λ x1_e: (Forall w2_s: ((w2_s << Des_<(e,s),{s}>(Alfonso_e, w_s)) >> Buy_<(e,e,s),t>(Alfonso_e, (ι x_e: ((Inexpensive_<(e,s),t>(x_e, w2_s) & Coat_<(e,s),t>(x_e, w2_s)) & (x_e <=> x1_e))), w2_s))))'/<e,t> conflict (mode: Function argument combination (unification failed))])

Basically, the $\iota$ condition on $x$s requires that any witness for the outer quantifier satisfy the properties inside the $\iota$ condition, and so the formula is only true if there are values of $x$ that do satisfy these conditions. (Is it false or undefined if not? As I understand it, the intent is that it would be false. The Russellian model-theoretic interpretation for inverted $\iota$ in, e.g., Gamut vol 1 sec 5.2 would derive the result needed.)

In conclusion: I hope that if fourth readings are real, there is another way to derive them.